### From MLG to nn

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn


In [101]:
df  = datasets.load_diabetes()

In [102]:
X = torch.from_numpy(df.data)
y = torch.from_numpy(df.target)

### Nothing new until now

In [103]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(X, y)

from torch.utils.data import DataLoader


train_loader = DataLoader(dataset, batch_size = 25, shuffle=True)


In [65]:
#https://www.analyticsvidhya.com/blog/2021/08/linear-regression-and-gradient-descent-in-pytorch/
def mse(y,y_hat):
    return torch.sum(torch.pow(y-y_hat,2))/torch.numel(y)



In [104]:
w = torch.randn(1,int(X[1].numel()), requires_grad=True)
b = torch.randn(int(X.numel()/X[1].numel()), 1, requires_grad=True)


In [105]:
def LinearReg(X):
    y = X.float() @ w.transpose(0,1) + b
    return y
    

In [107]:
#LinearReg(X)

mse(y, LinearReg(X))

tensor(12857654.2740, dtype=torch.float64, grad_fn=<DivBackward0>)

In [115]:
learning_rate = 1e-6
for i in range(1000):
    y_hat = LinearReg(X)
    loss = mse(y,y_hat)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad = None
        b.grad = None

In [116]:
y_hat = LinearReg(X)
torch.sqrt(mse(y,y_hat))

tensor(3518.2241, dtype=torch.float64, grad_fn=<SqrtBackward>)